# 🎓 SFT v6.1: Colab Curriculum Learning

**LFM2-1.2B 기반 한국어-영어 양방향 번역 SFT 학습**

> **환경**: Google Colab T4 GPU

## 사용 전 설정
1. Colab Secrets에 `HF_TOKEN` 등록
2. `OUTPUT_MODEL_ID`를 본인 HuggingFace ID로 변경

## 학습 전략
- **Base Model**: `gyung/lfm2-1.2b-koen-mt-v6-sft-200k` (Phase 1 완료)
- **Dataset**: HuggingFace `gyung/lfm2-koen-samples`
- **Method**: Curriculum Learning (고품질 데이터로 미세조정)

In [ ]:
# 1. 패키지 설치
!pip install -q transformers==4.54.0 trl>=0.18.2 peft>=0.15.2
!pip install -q datasets accelerate bitsandbytes huggingface_hub

print("✅ 설치 완료. 런타임 재시작이 필요할 수 있습니다.")

In [ ]:
# 2. HuggingFace 로그인
from huggingface_hub import login
login()  # Colab Secrets 사용 또는 수동 입력

In [ ]:
# 3. 라이브러리 임포트
import os
os.environ['WANDB_DISABLED'] = 'true'
os.environ['HF_HOME'] = '/tmp/hf_cache'  # 디스크 절약

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig, setup_chat_format

print(f"🔧 PyTorch: {torch.__version__}")
print(f"🔧 CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🖥️ GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# 4. 설정 (⚠️ OUTPUT_MODEL_ID는 본인 ID로 변경!)
MODEL_ID = "gyung/lfm2-1.2b-koen-mt-v6-sft-200k"  # Phase 1 모델
OUTPUT_MODEL_ID = "YOUR_ID/your-sft-model"  # ⚠️ 변경 필요!
DATASET_REPO = "gyung/lfm2-koen-samples"
DATASET_CONFIG = "manual_1000_sft"  # SFT용 데이터셋

print(f"📌 Base Model: {MODEL_ID}")
print(f"📌 Dataset: {DATASET_REPO} ({DATASET_CONFIG})")
print(f"📌 Output: {OUTPUT_MODEL_ID}")

In [ ]:
# 5. 모델 로드
print("🤖 모델 로딩 중...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Chat template 설정
if tokenizer.chat_template is None:
    model, tokenizer = setup_chat_format(model, tokenizer)

print(f"✅ 모델 로드 완료: {MODEL_ID}")

In [ ]:
# 6. 데이터셋 로드 (HuggingFace에서 직접)
print(f"📚 데이터셋 로딩: {DATASET_REPO} ({DATASET_CONFIG})")
dataset = load_dataset(DATASET_REPO, DATASET_CONFIG, split="train")
print(f"✅ 샘플 수: {len(dataset)}")

# 데이터 형식 확인
print("\n📋 샘플 예시:")
print(dataset[0])

In [ ]:
# 7. 데이터셋 전처리 (Chat Format 변환)
def format_to_chat(example):
    """
    SFT 데이터를 Chat Format으로 변환
    
    원본 형식: {instruction, input, output}
    변환 형식: {messages: [{role, content}, ...]}
    """
    messages = [
        {"role": "system", "content": example['instruction']},
        {"role": "user", "content": example['input']},
        {"role": "assistant", "content": example['output']}
    ]
    return {"messages": messages}

print("🔄 Chat Format 변환 중...")
train_dataset = dataset.map(format_to_chat)
print(f"✅ 변환 완료: {len(train_dataset)} samples")

# 변환 결과 확인
print("\n📋 변환된 샘플 예시:")
print(train_dataset[0]['messages'])

In [ ]:
# 8. SFT 학습 설정
sft_config = SFTConfig(
    output_dir="./sft-output",
    
    # 학습 파라미터
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    gradient_checkpointing=True,
    
    # 최적화
    learning_rate=5e-6,  # 낮은 LR (Curriculum Learning)
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    optim="paged_adamw_8bit",
    
    # 정밀도 (T4: FP16)
    fp16=True,
    bf16=False,
    
    # 로깅 및 저장
    logging_steps=50,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    
    # Hub 업로드
    push_to_hub=True,
    hub_model_id=OUTPUT_MODEL_ID,
    hub_strategy="every_save",
    
    # 데이터셋 설정
    dataset_text_field="messages",
    packing=False,
    report_to="none",
)

print("✅ 학습 설정 완료")
print(f"   Learning Rate: {sft_config.learning_rate}")
print(f"   Batch Size: {sft_config.per_device_train_batch_size} x {sft_config.gradient_accumulation_steps} = {sft_config.per_device_train_batch_size * sft_config.gradient_accumulation_steps}")

In [ ]:
# 9. Trainer 생성
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    args=sft_config,
    processing_class=tokenizer,
)

print("✅ Trainer 생성 완료")
print(f"   예상 학습 Step 수: {len(train_dataset) // (sft_config.per_device_train_batch_size * sft_config.gradient_accumulation_steps)}")

In [ ]:
# 10. 학습 시작!
print("🚀 학습 시작...")
print("="*50)

trainer.train()

print("="*50)
print("✅ 학습 완료!")

In [ ]:
# 11. 모델 저장 및 업로드
print("💾 모델 저장 및 업로드 중...")

trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")

trainer.push_to_hub()

print(f"✅ 업로드 완료: https://huggingface.co/{OUTPUT_MODEL_ID}")

In [ ]:
# 12. (선택) 학습된 모델 테스트
print("🧪 모델 테스트...")

# 테스트 문장
test_texts = [
    "Please fix this error.",
    "The model is working correctly now.",
    "Time flies like an arrow.",
]

for text in test_texts:
    messages = [
        {"role": "system", "content": "Translate to Korean."},
        {"role": "user", "content": text}
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages, 
        return_tensors="pt", 
        add_generation_prompt=True
    ).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.3,
        )
    
    result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    print(f"📝 EN: {text}")
    print(f"🇰🇷 KO: {result}")
    print("-" * 40)